The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install -U --pre mxnet-cu101mkl  # updating mxnet to at least v1.6


Requirement already up-to-date: mxnet-cu101mkl in /usr/local/lib/python3.6/dist-packages (1.6.0b20191006)


# GPUs

:label:`sec_use_gpu`


In the introduction to this book we discussed the rapid growth of computation over the past two decades. In a nutshell, GPU performance has increased by a factor of 1000 every decade since 2000. This offers great opportunity but it also suggests a significant need to provide such performance.

|Decade|Dataset|Memory|Floating Point Calculations per Second|
|:--|:-|:-|:-|
|1970|100 (Iris)|1 KB|100 KF (Intel 8080)|
|1980|1 K (House prices in Boston)|100 KB|1 MF (Intel 80186)|
|1990|10 K (optical character recognition)|10 MB|10 MF (Intel 80486)|
|2000|10 M (web pages)|100 MB|1 GF (Intel Core)|
|2010|10 G (advertising)|1 GB|1 TF (NVIDIA C2050)|
|2020|1 T (social network)|100 GB|1 PF (NVIDIA DGX-2)|

In this section we begin to discuss how to harness this compute performance for your research. First by using single GPUs and at a later point, how to use multiple GPUs and multiple servers (with multiple GPUs). You might have noticed that MXNet `ndarray` looks almost identical to NumPy. But there are a few crucial differences. One of the key features that differentiates MXNet from NumPy is its support for diverse hardware devices.

In MXNet, every array has a context. In fact, whenever we displayed an `ndarray` so far, it added a cryptic `@cpu(0)` notice to the output which remained unexplained so far. As we will discover, this just indicates that the computation is being executed on the CPU. Other contexts might be various GPUs. Things can get even hairier when we deploy jobs across multiple servers. By assigning arrays to contexts intelligently, we can minimize the time spent transferring data between devices. For example, when training neural networks on a server with a GPU, we typically prefer for the model’s parameters to live on the GPU.

In short, for complex neural networks and large-scale data, using only CPUs for computation may be inefficient. In this section, we will discuss how to use a single NVIDIA GPU for calculations. First, make sure you have at least one NVIDIA GPU installed. Then, [download CUDA](https://developer.nvidia.com/cuda-downloads) and follow the prompts to set the appropriate path. Once these preparations are complete, the `nvidia-smi` command can be used to view the graphics card information.

In [2]:
!nvidia-smi

Thu Feb 27 02:57:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Next, we need to confirm that the GPU version of MXNet is installed. If a CPU version of MXNet is already installed, we need to uninstall it first. For example, use the `pip uninstall mxnet` command, then install the corresponding MXNet version according to the CUDA version. Assuming you have CUDA 9.0 installed, you can install the MXNet version that supports CUDA 9.0 by `pip install mxnet-cu90`. To run the programs in this section, you need at least two GPUs.

Note that this might be extravagant for most desktop computers but it is easily available in the cloud, e.g., by using the AWS EC2 multi-GPU instances. Almost all other sections do *not* require multiple GPUs. Instead, this is simply to illustrate how data flows between different devices.

## Computing Devices

MXNet can specify devices, such as CPUs and GPUs, for storage and calculation. By default, MXNet creates data in the main memory and then uses the CPU to calculate it. In MXNet, the CPU and GPU can be indicated by `cpu()` and `gpu()`. It should be noted that `cpu()` (or any integer in the parentheses) means all physical CPUs and memory. This means that MXNet's calculations will try to use all CPU cores. However, `gpu()` only represents one graphic card and the corresponding graphic memory. If there are multiple GPUs, we use `gpu(i)` to represent the $i^\mathrm{th}$ GPU ($i$ starts from 0). Also, `gpu(0)` and `gpu()` are equivalent.

In [3]:
from mxnet import np, npx
from mxnet.gluon import nn
npx.set_np()

npx.cpu(), npx.gpu(), npx.gpu(1)

(cpu(0), gpu(0), gpu(1))

In [4]:
help(npx)

Help on package mxnet.numpy_extension in mxnet:

NAME
    mxnet.numpy_extension - Module for ops not belonging to the official numpy package for imperative programming.

PACKAGE CONTENTS
    _op
    _register
    image
    random
    utils

DATA
    __all__ = []

FILE
    /usr/local/lib/python3.6/dist-packages/mxnet/numpy_extension/__init__.py




In [5]:
help(np)

Help on package mxnet.numpy in mxnet:

NAME
    mxnet.numpy - MXNet NumPy module.

PACKAGE CONTENTS
    _op
    _register
    arrayprint
    function_base
    io
    linalg
    multiarray
    random
    stride_tricks
    utils

DATA
    __all__ = []

FILE
    /usr/local/lib/python3.6/dist-packages/mxnet/numpy/__init__.py




We can query the number of available GPUs through `num_gpus()`.

In [6]:
npx.num_gpus()

1

Now we define two convenient functions that allows us to run codes even if the requested GPUs do not exist.

In [7]:
# Saved in the d2l package for later use
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    return npx.gpu(i) if npx.num_gpus() >= i + 1 else npx.cpu()

# Saved in the d2l package for later use
def try_all_gpus():
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    ctxes = [npx.gpu(i) for i in range(npx.num_gpus())]
    return ctxes if ctxes else [npx.cpu()]

try_gpu(), try_gpu(3), try_all_gpus()

(gpu(0), cpu(0), [gpu(0)])

## `ndarray` and GPUs

By default, `ndarray` objects are created on the CPU. Therefore, we will see the `@cpu(0)` identifier each time we print an `ndarray`.

In [8]:
x = np.array([1, 2, 3])
x

array([1., 2., 3.])

We can use the `context` property of `ndarray` to view the device where the `ndarray` is located. It is important to note that whenever we want to operate on multiple terms they need to be in the same context. For instance, if we sum two variables, we need to make sure that both arguments are on the same device---otherwise MXNet would not know where to store the result or even how to decide where to perform the computation.

In [9]:
x.context

cpu(0)

### Storage on the GPU

There are several ways to store an `ndarray` on the GPU. For example, we can specify a storage device with the `ctx` parameter when creating an `ndarray`. Next, we create the `ndarray` variable `a` on `gpu(0)`. Notice that when printing `a`, the device information becomes `@gpu(0)`. The `ndarray` created on a GPU only consumes the memory of this GPU. We can use the `nvidia-smi` command to view GPU memory usage. In general, we need to make sure we do not create data that exceeds the GPU memory limit.

In [10]:
x = np.ones((2, 3), ctx=try_gpu())
x

array([[1., 1., 1.],
       [1., 1., 1.]], ctx=gpu(0))

Assuming you have at least two GPUs, the following code will create a random array on `gpu(1)`.

In [11]:
y = np.random.uniform(size=(2, 3), ctx=try_gpu(1))
y

array([[0.5488135 , 0.5928446 , 0.71518934],
       [0.84426576, 0.60276335, 0.8579456 ]])

In [12]:
y.ctx

cpu(0)

In [13]:
y = np.random.uniform(size=(2, 3), ctx=try_gpu(0))
y.ctx

gpu(0)

### Copying

If we want to compute $\mathbf{x} + \mathbf{y}$ we need to decide where to perform this operation. For instance, as shown in :numref:`fig_copyto`, we can transfer $\mathbf{x}$ to `gpu(1)` and perform the operation there. *Do not* simply add `x + y` since this will result in an exception. The runtime engine would not know what to do, it cannot find data on the same device and it fails.

![Copyto copies arrays to the target device](http://d2l.ai/_images/copyto.svg)

:label:`fig_copyto`


`copyto` copies the data to another device such that we can add them. Since $\mathbf{y}$ lives on the second GPU we need to move $\mathbf{x}$ there before we can add the two.

In [14]:
z = x.copyto(try_gpu(0))
print(x)
print(z)

[[1. 1. 1.]
 [1. 1. 1.]] @gpu(0)
[[1. 1. 1.]
 [1. 1. 1.]] @gpu(0)


Now that the data is on the same GPU (both $\mathbf{z}$ and $\mathbf{y}$ are), we can add them up. In such cases MXNet places the result on the same device as its constituents. In our case that is `@gpu(1)`.

In [15]:
y + z

array([[1.7402194, 1.9209938, 1.0390205],
       [1.9689629, 1.9251406, 1.4463501]], ctx=gpu(0))

In [16]:
z.copyto(npx.cpu())
z.ctx

gpu(0)

In [17]:
w = z.copyto(npx.cpu())
w.ctx

cpu(0)

In [18]:
w = w.as_in_context(try_gpu(1))
z

array([[1., 1., 1.],
       [1., 1., 1.]], ctx=gpu(0))

Imagine that your variable z already lives on your second GPU (gpu(1)). What happens if we call z.copyto(gpu(1))? It will make a copy and allocate new memory, even though that variable already lives on the desired device!
There are times where depending on the environment our code is running in, two variables may already live on the same device. So we only want to make a copy if the variables currently lives on different contexts. In these cases, we can call `as_in_context()`. If the variable is already the specified context then this is a no-op. In fact, unless you specifically want to make a copy, `as_in_context()` is the method of choice.

In [19]:
z = x.as_in_context(try_gpu(1))
z.ctx

cpu(0)

It is important to note that, if the `context` of the source variable and the target variable are consistent, then the `as_in_context` function causes the target variable and the source variable to share the memory of the source variable.

In [20]:
y.ctx

gpu(0)

In [21]:
y.as_in_context(try_gpu(0)) is y # why doesn't this work

False

In [22]:
x.ctx

gpu(0)

In [23]:
x.as_in_context(try_gpu(0)) is x # still doesn't work

False

In [24]:
hex(id(x.as_in_context(npx.gpu(0)))), hex(id(x))

('0x7f868e664090', '0x7f870bf62e58')

The `copyto` function always creates new memory for the target variable.

In [25]:
y.copyto(try_gpu(1)) is y

False

### Side Notes

People use GPUs to do machine learning because they expect them to be fast. But transferring variables between contexts is slow. So we want you to be 100% certain that you want to do something slow before we let you do it. If MXNet just did the copy automatically without crashing then you might not realize that you had written some slow code.

Also, transferring data between devices (CPU, GPUs, other machines) is something that is *much slower* than computation. It also makes parallelization a lot more difficult, since we have to wait for data to be sent (or rather to be received) before we can proceed with more operations. This is why copy operations should be taken with great care. As a rule of thumb, many small operations are much worse than one big operation. Moreover, several operations at a time are much better than many single operations interspersed in the code (unless you know what you are doing). This is the case since such operations can block if one device has to wait for the other before it can do something else. It is a bit like ordering your coffee in a queue rather than pre-ordering it by phone and finding out that it is ready when you are.

Last, when we print `ndarray`s or convert `ndarray`s to the NumPy format, if the data is not in main memory, MXNet will copy it to the main memory first, resulting in additional transmission overhead. Even worse, it is now subject to the dreaded Global Interpreter Lock which makes everything wait for Python to complete.


## Gluon and GPUs

Similarly, Gluon's model can specify devices through the `ctx` parameter during initialization. The following code initializes the model parameters on the GPU (we will see many more examples of how to run models on GPUs in the following, simply since they will become somewhat more compute intensive).

In [0]:
net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(ctx=try_gpu())

When the input is an `ndarray` on the GPU, Gluon will calculate the result on the same GPU.

In [27]:
net(x)

array([[0.04421056],
       [0.04421056]], ctx=gpu(0))

Let's confirm that the model parameters are stored on the same GPU.

In [28]:
net[0].weight.data()

array([[ 0.00628365,  0.04861524, -0.01068833]], ctx=gpu(0))

In short, as long as all data and parameters are on the same device, we can learn models efficiently. In the following we will see several such examples.

## Summary

* MXNet can specify devices for storage and calculation, such as CPU or GPU. By default, MXNet creates data in the main memory and then uses the CPU to calculate it.
* MXNet requires all input data for calculation to be *on the same device*, be it CPU or the same GPU.
* You can lose significant performance by moving data without care. A typical mistake is as follows: computing the loss for every minibatch on the GPU and reporting it back to the user on the command line (or logging it in a NumPy array) will trigger a global interpreter lock which stalls all GPUs. It is much better to allocate memory for logging inside the GPU and only move larger logs.

## Exercises

1. Try a larger computation task, such as the multiplication of large matrices, and see the difference in speed between the CPU and GPU. What about a task with a small amount of calculations?
1. How should we read and write model parameters on the GPU?
1. Measure the time it takes to compute 1000 matrix-matrix multiplications of $100 \times 100$ matrices and log the matrix norm $\mathrm{tr} M M^\top$ one result at a time vs. keeping a log on the GPU and transferring only the final result.
1. Measure how much time it takes to perform two matrix-matrix multiplications on two GPUs at the same time vs. in sequence on one GPU (hint: you should see almost linear scaling).

## [Discussions](https://discuss.mxnet.io/t/2330)

![](https://github.com/d2l-ai/d2l-en-colab/blob/master/img/qr_use-gpu.svg?raw=1)

In [0]:
def multiply_matrices(ctx, size, times):
    A = np.random.uniform(size=size, ctx=ctx)
    B = np.random.uniform(size=size, ctx=ctx)
    for i in range(times):
        B = np.dot(A, B)
    return B.as_in_context(npx.cpu())

In [0]:
def multiply_matrices(A, B, ctx, times):
    A = A.as_in_context(ctx)
    B = B.as_in_context(ctx)
    for i in range(times):
        B = np.dot(A, B)
    return B.as_in_context(npx.cpu())

In [0]:
contexts = {'cpu': npx.cpu(), 'gpu': npx.gpu(0)}

In [32]:
A = np.random.uniform(size=[10, 10])
B = np.random.uniform(size=[10, 10])

import time
times = {}
for ctx_name, ctx in contexts.items():
    start = time.time()
    res = multiply_matrices(A, B, ctx, times=1)
    times[ctx_name] = time.time() - start
print(times)

{'cpu': 0.0009286403656005859, 'gpu': 0.0011246204376220703}


In [89]:
  A = np.random.uniform(size=[5000, 5000])
  B = np.random.uniform(size=[5000, 5000])

import time
times = {}
for ctx_name, ctx in contexts.items():
    start = time.time()
    res = multiply_matrices(A, B, ctx, times=100000)
    times[ctx_name] = time.time() - start
print(times)

{'cpu': 2.0949666500091553, 'gpu': 1.6278984546661377}


In [90]:
  A = np.random.uniform(size=[5000, 5000])
  B = np.random.uniform(size=[5000, 5000])

import time
times = {}
for ctx_name, ctx in contexts.items():
    start = time.time()
    for i in range(100000):
      res = multiply_matrices(A, B, ctx, times=1)
    times[ctx_name] = time.time() - start
print(times)

{'cpu': 6.495464324951172, 'gpu': 26.09580421447754}


In [56]:
A = np.random.uniform(size=[100, 100], ctx=npx.gpu(0))
start = time.time()
res = np.random.uniform(size=[1000, 1], ctx=npx.gpu(0))
#print(res.ctx)
#res = np.zeros(size=[1000, 1], ctx=npx.gpu())
for i in range(1000):
  res[i] = np.log(np.trace(np.dot(A, A.T)))
#print(A.ctx)
a = res.as_in_context(npx.cpu())
total = time.time() - start
print(total)

0.34454989433288574


In [57]:
A = np.random.uniform(size=[100, 100], ctx=npx.gpu(0))
start = time.time()
res = np.random.uniform(size=[1000, 1], ctx=npx.cpu())
#print(res.ctx)
for i in range(1000):
  res[i] = np.log(np.trace(np.dot(A, A.T))).as_in_context(npx.cpu())
total = time.time() - start
print(total)

0.4237484931945801
